In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
%matplotlib inline
from pandas_datareader import data as pdr
from datetime import datetime,date
import yfinance as yfin
import requests
import scipy.optimize as sco
import scipy.interpolate as sci
import random

In [ ]:
data = pd.read_csv('XXXX.csv', index_col=0, parse_dates =[2]).dropna()

In [ ]:
data

In [ ]:
def year_trans(data):
    data = data.groupby([data.DlyCalDt.dt.year,'Ticker'])['IssuerNm'].count().unstack(level = 'DlyCalDt')
    return(data)

def mode_trans(data):
    for i in data.columns:
        data[i] = data[i].where(data[i] == data[i].mode()[0],float('NaN'))
    return(data)

def weight_gen(port_num):
    p_weights = []
    weights = np.random.random(size = port_num)
    weights = weights / np.sum(weights)
    p_weights.append(weights)
    return p_weights[0].tolist()

def tday_filter (dataog,datafilter,year):
    mask1 = dataog['DlyCalDt'].dt.year == year
    datanew = dataog[mask1]
    mask2 = datanew.index.isin(datafilter[year].dropna().index)
    datanew2 = datanew[mask2]
    return datanew2

def random_select(universe,portsize):
    random_port = random.sample(list(universe.index.unique()),portsize)
    return random_port

#returns portfolio yearly return
def return_gen(data,assets,weights):
    m = data.index.isin(assets)
    data = data[m]
    ret = 0
    for i,j in zip(data.index.unique(),weights):
        ret = ret + data.loc[i,'DlyRet'].sum()*j

    return ret

#returns individual daily returns for items in portfolio
def return_gen2(data,assets,weights):
    empty = pd.DataFrame()
    m = data.index.isin(assets)
    data = data[m]
    ret = 0
    data2 = data.set_index(data['DlyCalDt'].dt.date,append = True)
    for i,j in zip(data2.index.get_level_values('Ticker').unique(),weights):
        test = data2.loc[i,'DlyRet'] * j
        empty[i]=test
    return empty

def daily_return(data):
    row_sum = data.sum(axis=1)
    return row_sum

def vol(daily_returns,annual_return):
    sd = ((((daily_returns - (annual_return/len(daily_returns)))**2).sum())*(len(daily_returns)/(len(daily_returns)-1)))**0.5
    return sd

In [ ]:
data1=year_trans(data)
data1t = mode_trans(data1)
data1

In [ ]:
N = 100
yr = XXXX
x = tday_filter(data,data1t,yr)
weight = weight_gen(N)
z = random_select(x,N)
ret = return_gen(x,z,weight)
ret

In [ ]:
ret2 = return_gen2(x,z,weight)
ret2

In [ ]:
dret = daily_return(ret2)
dret

In [ ]:
sd = vol(dret,ret)
sd

In [ ]:
#calculate fama french risk free rate
def fama_french(portfolio):
    high_low = sorted(portfolio)
    num_trades = len(portfolio)
    r_f = (((high_low[0] + high_low[num_trades - 2])/2)*num_trades)
    return r_f

print(fama_french(dret))

#cml line assuming volatility is x axis and return is y
def cml(annual_return, interest, vol):
    cml_int = interest
    cml_slope = ((annual_return + interest)/vol)
    return cml_int, cml_slope

In [ ]:
#generate the random distribution
x = []
y = []
ret2_df = pd.DataFrame(ret2)
for column in ret2_df:
    stock = ret2_df[column]
    y.append(sum(stock)*(N/10))
    x.append(vol(stock, ret)*(N/10))


In [ ]:
#create the plot
cml_line = cml(ret, fama_french(dret), sd)
m = cml_line[1]
b = cml_line[0]
fig, ax = plt.subplots()
ax.scatter(x, y, c='blue')
ax.axline((0, b), slope=-m, color='black', label='cml')
ax.legend()
plt.suptitle("Random Portfolios Year XXXX")
plt.title("N = 100")
plt.show()